In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# <b><font color="navy">Regresion lineal</span></font></b>

## 0. Replicar ejercicio Excel

### 0.1 Cargar datos ejercicio excel

In [ ]:
ruta_datos_excel = r'dataset\datos_ejercicio_regresion_lineal.xlsx'
df_sm = pd.read_excel(ruta_datos_excel )
df_sm

In [ ]:
# Adicionar Variable Dummy
def colegio_en_clase(columna):
    if columna == 'Si':
        return 1
    else:
        return 0
df_sm['Colegio_Abierto_Dummy'] =  df_sm['Colegio Abierto'].apply(colegio_en_clase)
df_sm

### 0.2 Aplicar modelo regresion lineal desde statsmodel

**Ejercicio:** Usar prompt en ChatGPT

## 1. Cargar datos

In [ ]:
# Definir ubicacion datos
ruta_demanda = r'dataset\demanda_dia.csv'

# Leer archivo
df = pd.read_csv(ruta_demanda)

## 2. Pre-procesamiento de datos

In [ ]:
# Asignar formato fecha a la columna FECHA
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d-%m-%y')

# Establecer FECHA como datetime index
df.set_index('FECHA', inplace=True)

# Filtrar fechas mayores a 2021-01-01
df = df[df.index >= '2021-01-01'].copy()
df

### 2.1 Llenar todas las fechas faltantes con 0

In [ ]:
# Tabla dinamica para llenar fechas faltantes con cero´s
df = pd.pivot_table(data=df,
                         values='DEMANDA',
                         index=['COD_SKU','DESC_SKU'],
                         columns='FECHA',
                         aggfunc='sum',
                         fill_value=0).stack().reset_index() 

# Devolver nombre a columna de demanda ya que se pierde al hacer la tabla
df = df.rename(columns = {0:'DEMANDA'})

# Volver a asignar fecha como indice
df = df.set_index(df['FECHA'])
df

## 3. Feature Engineering

### 3.1 Atributos a extraer desde la fecha

Son atributos que pueden extraerse desde la fecha de cada registro. Pueden ser entre otros: día de la semana, número de día, número
del mes, número del año, número de trimestre, días laborales, si es fin de semana, si es fin de mes, si es inicio de mes, número de días del mes

In [ ]:
# Extrae el dia de la semana del index
df['DIA_SEM'] = df.index.weekday

# Extrae el Mes del index
df['MES'] = df.index.month

# Extrae el dia del mes del index
df['DIA'] = df.index.day

# Extrae el numero de la semana del index
df['SEMANA'] = df.index.isocalendar().week

# Extrae el numero del año del index
df['AÑO'] = df.index.year

df

### 3.2 Variable Dummy Festivos

In [ ]:
# Importar libreria de festivos
import holidays

# Descargar festivos Colombia
festivos_colombia = holidays.CO()

# Filtrar por años necesarios
festivos_colombia = [items[0] for items in holidays.CO(years=[2021, 2022, 2023]).items()]

#festivos_colombia

In [ ]:
# Adiciona un 1 si el index esta en festivo_colombia, si no esta, adiciona 0
df['FESTIVO'] = df.index.isin(pd.to_datetime(festivos_colombia)).astype(int)

df

### 3.3 Atributos tipo "LAGS"

Son "desfases" que se usan como atributos y que pueden aportar significativamente a que el algoritmo detecte la estacionalidad en la serie de tiempo. Primero debe agruparse la serie de tiempo por SKU y luego aplicar el desfase con la función apply lambda. Debe tenerse cuidado de que no se generen desfases entre SKU´s

In [ ]:
# Adiciona una columna con la demanda desfasada 7 periodos
df['DEMANDA_LAG_7'] = df.groupby(['COD_SKU', 'DESC_SKU'])['DEMANDA'].shift(7)

df

# <b><font color="navy">Modelo 1</span></font></b>

## 4. Seleccionar SKU

In [ ]:
SKU = 'PT-013'

In [ ]:
# Seleccionar Referencia
df_modelo_1 = df[df['COD_SKU'] == SKU].copy()

# Eliminar NaN
df_modelo_1 = df_modelo_1.dropna(subset = 'DEMANDA_LAG_7')

df_modelo_1

## 5. Seleccionar Atributos y target

In [ ]:
# Seleccionar variables atributos
atributos = ['DIA_SEM',	'MES',	'DIA',	'SEMANA',	'AÑO',	'FESTIVO',	'DEMANDA_LAG_7']

# Separar atributos en X y target en y
X = df_modelo_1[atributos]
y = df_modelo_1['DEMANDA']

## 6. Set de entrenamiento y Set de testeo

In [ ]:
# Dividir el set de datos: 80% Entrenamiento - 20% Testeo
punto_limite = int(len(df_modelo_1) * 0.8)

# Separar Set de atributos y target por fecha limite
X_train, X_test = X[:punto_limite], X[punto_limite:]
y_train, y_test = y[:punto_limite], y[punto_limite:]

print("X_train:")
print(X_train)
print("\nX_test:")
print(X_test)
print("\ny_train:")
print(y_train)
print("\ny_test:")
print(y_test)

## 7. Entrenamiento del modelo

In [ ]:
# Importar Regresion Lineal de scikitlearn
from sklearn.linear_model import LinearRegression


In [ ]:
# Entrenar el modelo
model_1 = LinearRegression()
model_1.fit(X_train, y_train)

# Predecir en el set de testeo
y_pred = model_1.predict(X_test)


## 8. Función de métricas de error KPIs

In [ ]:
def kpi(y_test, y_pred):
    
    # Aplicar estilo ggplot
    plt.style.use('ggplot')
    # Calcula MAE%
    mae_porc = (abs(y_test - y_pred).sum())/y_test.sum()
    # Calcula Sesgo
    sesgo = (y_test - y_pred).sum()/y_test.sum()
    # Muestra el MAE%
    print('MAE%: {:.2%}'.format(mae_porc)) 
    # Muestra el sesgo%
    print('Sesgo: {:.2%}'.format(sesgo)) 
    # Calcula el score
    score = mae_porc + abs(sesgo)
    # Muestra el score
    print('Score: {:.2%}'.format(score))

    #Grafica
    y_test_df = pd.DataFrame(y_test)
    
    # Graficar Demanda real vs Pronóstico
    plt.figure(figsize=(8, 3))
    plt.plot(y_test_df.index, y_test_df, label='Demanda', )
    plt.plot(y_test_df.index, y_pred, label='Pronóstico RL',  linestyle='--')
    plt.xlabel('Periodo', fontsize=10)
    plt.xticks(rotation=30, fontsize=7)
    plt.ylabel('Cantidad', fontsize=10)
    plt.title('Demanda vs Pronóstico')
    plt.legend()
    plt.show()


In [ ]:
kpi(y_test, y_pred)

In [ ]:
def importancia_atributos(model, X_test, y_test):

    # Extrae el coeficiente de cada atributo del resumen de scikitlearn
    importancia_atributos = np.abs(model.coef_)
    nombres_atributos = X_train.columns
    
    # Ordenar desdendentemente por importancia
    indice_ordenado = np.argsort(importancia_atributos)[::-1]
    importancia_ordenado = importancia_atributos[indice_ordenado]
    atributos_ordenados = nombres_atributos[indice_ordenado]
    
    # Caluculo de R cuadrado 
    r_cuadrado = model.score(X_test, y_test)
    print('R-cuadrado: {:.2%}'.format(r_cuadrado))
    
    # Calculo de R Cuadrado Ajustado
    n = len(y_test)
    k = X_test.shape[1]
    r_cuadrado_ajustado = 1 - (1 - r_cuadrado) * (n - 1) / (n - k - 1)
    print('R-cuadrado ajustado: {:.2%}'.format(r_cuadrado_ajustado))
    
    # Gráfica
    plt.figure(figsize=(6, 3))
    plt.bar(range(len(importancia_ordenado)), importancia_ordenado, align='center')
    plt.xticks(range(len(importancia_ordenado)), atributos_ordenados, rotation=30, fontsize=7)
    plt.ylabel('Importancia (Magnitud Coeficiente)', fontsize=8)
    plt.title('Importancia de Atributos en Regresion Lineal',fontsize=12)
    plt.show()

In [ ]:
importancia_atributos(model_1, X_test, y_test)

# <b><font color="navy">Modelo 2</span></font></b>

## 9. Atributos de PMS estacional y Seno de Dia_Sem

In [ ]:
# Retomar datos iniciales
df_modelo_2 = df.copy()

# Adicionar Promedio movil simple con n=3 de forma estacional
df_modelo_2['PMS_DEMANDA_3'] = df_modelo_2.groupby(['COD_SKU', 'DESC_SKU', 'DIA_SEM'])['DEMANDA'].transform(lambda x: x.rolling(window=3, 
                                                                                        min_periods=1).mean().shift(1))

# Adicionar Seno del Dia de la semana
df_modelo_2['SENO_DIA_SEM'] = np.sin(df_modelo_2['DIA_SEM'])

df_modelo_2

In [ ]:
# Seleccionar Referencia
df_modelo_2 = df_modelo_2[df_modelo_2['COD_SKU'] == SKU].copy()

# Eliminar NaN
df_modelo_2 = df_modelo_2.dropna(subset = ['DEMANDA_LAG_7','PMS_DEMANDA_3'])

df_modelo_2

In [ ]:
# Seleccionar variables atributos
atributos = ['DIA_SEM', 'MES',	'DIA',	'SEMANA',	'AÑO',	'FESTIVO', 'DEMANDA_LAG_7',  'PMS_DEMANDA_3', 
            'SENO_DIA_SEM',	]

# Separar atributos en X y target en y
X = df_modelo_2[atributos]
y = df_modelo_2['DEMANDA']

In [ ]:
# Dividir el set de datos: 80% Entrenamiento - 20% Testeo
punto_limite = int(len(df_modelo_2) * 0.8)

# Separar Set de atributos y target por fecha limite
X_train, X_test = X[:punto_limite], X[punto_limite:]
y_train, y_test = y[:punto_limite], y[punto_limite:]

In [ ]:
# Entrenar el modelo 2
model_2 = LinearRegression()
model_2.fit(X_train, y_train)

# Predecir en el set de testeo
y_pred = model_2.predict(X_test)


In [ ]:
kpi(y_test, y_pred)

In [ ]:
importancia_atributos(model_2, X_test, y_test)

# <b><font color="navy">Modelo 3</span></font></b>

## 10. Mejorar atributo de demanda promedio incluyendo los festivos

In [ ]:
df_modelo_3 = df.copy()

# Calcula el promedio movil simple de los ultimos 3 periodos estacionales incluyendo si es festivo
df_modelo_3['PMS_DEMANDA_3'] = df_modelo_3.groupby(['COD_SKU', 'DESC_SKU', 'DIA_SEM', 'FESTIVO'])['DEMANDA'].transform(lambda x: x.rolling(window=3, 
                                                                                        min_periods=1).mean().shift(1))

# Adiciona el Seno del dia de la semana
df_modelo_3['SENO_DIA_SEM'] = np.sin(df_modelo_3['DIA_SEM'])


In [ ]:
# Seleccionar Referencia
df_modelo_3 = df_modelo_3[df_modelo_3['COD_SKU'] == SKU].copy()

# Eliminar NaN
df_modelo_3 = df_modelo_3.dropna(subset = ['DEMANDA_LAG_7','PMS_DEMANDA_3'])

df_modelo_3

In [ ]:
# Seleccionar variables atributos
features = ['DIA_SEM', 'MES',	'DIA',	'SEMANA',	'AÑO',	'FESTIVO', 'DEMANDA_LAG_7',  'PMS_DEMANDA_3', 
            'SENO_DIA_SEM',	]

# Separar atributos en X y target en y
X = df_modelo_3[features]
y = df_modelo_3['DEMANDA']

In [ ]:
# Dividir el set de datos: 80% Entrenamiento - 20% Testeo
punto_limite = int(len(df_modelo_3) * 0.8)

# Separar Set de atributos y target por fecha limite
X_train, X_test = X[:punto_limite], X[punto_limite:]
y_train, y_test = y[:punto_limite], y[punto_limite:]

In [ ]:
# Entrenar modelo 3
model_3 = LinearRegression()
model_3.fit(X_train, y_train)

# Predict on the test set
y_pred = model_3.predict(X_test)


In [ ]:
kpi(y_test, y_pred)

In [ ]:
importancia_atributos(model_3, X_test, y_test)

# <b><font color="navy">Ejercicio</span></font></b>

Construya un modelo 4 cambiando algunos de los parametros e incluyendo nuevos atributos como:
- Coseno dia_sem
- Seno, Coseno dia del mes, mes, año
- Lags de 14, 21 ...   días
- Lag de 1 día
- Otros que usted considere